# Chapter 51: Microservices Integration

As applications scale, monolithic architectures often become bottlenecks for team autonomy and deployment velocity. Microservices decompose applications into independently deployable services aligned with business domains. This chapter explores patterns for integrating Next.js applications with microservice architectures—handling cross-service communication, maintaining data consistency, and managing the operational complexity of distributed systems.

By the end of this chapter, you'll decompose monoliths using domain-driven design boundaries, implement inter-service communication with REST, gRPC, and message brokers, architect Backend-for-Frontend (BFF) layers with Next.js API routes, manage distributed authentication with JWT and service mesh patterns, handle data consistency using Saga orchestration, implement resilience patterns like circuit breakers, and observe distributed systems with correlated logging and tracing.

## 51.1 Microservices Architecture

Decomposing applications into bounded contexts.

### Domain-Driven Decomposition

Identifying service boundaries using strategic DDD patterns:

```typescript
// Bounded Contexts in a typical e-commerce platform
const boundedContexts = {
  // Core Domains
  catalog: {
    entities: ['Product', 'Category', 'Inventory', 'Pricing'],
    responsibilities: 'Product information, search, availability',
    team: 'Catalog Team',
  },
  orders: {
    entities: ['Order', 'OrderItem', 'Fulfillment'],
    responsibilities: 'Order lifecycle, state management',
    team: 'Order Management Team',
  },
  payments: {
    entities: ['Payment', 'Refund', 'Transaction'],
    responsibilities: 'Payment processing, fraud detection',
    team: 'Payments Team',
  },
  users: {
    entities: ['User', 'Profile', 'Preferences', 'Authentication'],
    responsibilities: 'Identity, profiles, authorization',
    team: 'Identity Team',
  },
  notifications: {
    entities: ['Notification', 'Template', 'Delivery'],
    responsibilities: 'Email, SMS, push notifications',
    team: 'Communications Team',
  },
};

// Anti-corruption Layer (ACL) Pattern
// lib/microservices/acl/catalog-adapter.ts
interface CatalogProduct {
  sku: string;
  name: string;
  price: number;
  available: boolean;
}

export class CatalogServiceAdapter {
  private baseUrl: string;
  
  constructor() {
    this.baseUrl = process.env.CATALOG_SERVICE_URL!;
  }
  
  async getProduct(sku: string): Promise<CatalogProduct> {
    // Translate external API to internal domain model
    const response = await fetch(`${this.baseUrl}/products/${sku}`, {
      headers: { 'X-API-Key': process.env.CATALOG_API_KEY! },
      next: { revalidate: 60 },
    });
    
    if (!response.ok) {
      throw new CatalogServiceError(`Product ${sku} not found`);
    }
    
    const external = await response.json();
    
    // Map external schema to internal domain model
    return {
      sku: external.product_code,
      name: external.display_name,
      price: external.unit_price.amount,
      available: external.inventory.status === 'IN_STOCK',
    };
  }
  
  // Circuit breaker wrapper
  async getProductWithFallback(sku: string): Promise<CatalogProduct | null> {
    return withCircuitBreaker(
      () => this.getProduct(sku),
      {
        fallback: async () => {
          // Return from cache or default
          return getCachedProduct(sku);
        },
        failureThreshold: 5,
        resetTimeout: 30000,
      }
    );
  }
}
```

### Service Registry and Discovery

Dynamic service location in cloud-native environments:

```typescript
// lib/microservices/discovery.ts
interface ServiceInstance {
  id: string;
  name: string;
  host: string;
  port: number;
  health: 'healthy' | 'unhealthy';
  metadata: Record<string, string>;
  lastChecked: Date;
}

// Consul-based service discovery
export class ServiceDiscovery {
  private consul: any; // Consul client
  
  async discover(serviceName: string): Promise<ServiceInstance[]> {
    // In production, query Consul, Eureka, or Kubernetes DNS
    const services = await this.consul.health.service(serviceName);
    
    return services.map((s: any) => ({
      id: s.Service.ID,
      name: s.Service.Service,
      host: s.Service.Address,
      port: s.Service.Port,
      health: s.Checks.every((c: any) => c.Status === 'passing') ? 'healthy' : 'unhealthy',
      metadata: s.Service.Meta,
      lastChecked: new Date(),
    }));
  }
  
  // Load balancing strategy
  async getHealthyInstance(serviceName: string): Promise<ServiceInstance> {
    const instances = await this.discover(serviceName);
    const healthy = instances.filter(i => i.health === 'healthy');
    
    if (healthy.length === 0) {
      throw new Error(`No healthy instances for ${serviceName}`);
    }
    
    // Round-robin selection
    const index = Math.floor(Math.random() * healthy.length);
    return healthy[index];
  }
}

// Kubernetes DNS-based discovery (simpler alternative)
export function getServiceUrl(serviceName: string, namespace: string = 'default'): string {
  // In Kubernetes: http://service-name.namespace.svc.cluster.local
  return `http://${serviceName}.${namespace}.svc.cluster.local`;
}
```

## 51.2 Service Communication

Patterns for synchronous and asynchronous inter-service communication.

### REST and HTTP Clients

Type-safe HTTP clients with resilience:

```typescript
// lib/microservices/http/client.ts
import { z } from 'zod';

interface HttpClientConfig {
  baseUrl: string;
  timeout?: number;
  retries?: number;
  headers?: Record<string, string>;
}

export class ServiceHttpClient {
  private config: HttpClientConfig;
  
  constructor(config: HttpClientConfig) {
    this.config = {
      timeout: 5000,
      retries: 3,
      ...config,
    };
  }
  
  async get<T extends z.ZodType>(
    path: string,
    schema: T,
    options?: { cache?: boolean; revalidate?: number }
  ): Promise<z.infer<T>> {
    const url = new URL(path, this.config.baseUrl);
    
    const response = await fetch(url, {
      headers: {
        'Content-Type': 'application/json',
        ...this.config.headers,
      },
      next: options?.cache !== false ? {
        revalidate: options?.revalidate ?? 60,
      } : undefined,
    });
    
    if (!response.ok) {
      throw new ServiceError(response.status, await response.text());
    }
    
    const data = await response.json();
    return schema.parse(data); // Runtime validation
  }
  
  async post<T extends z.ZodType>(
    path: string,
    body: unknown,
    schema: T
  ): Promise<z.infer<T>> {
    // Implementation with retry logic
    let lastError: Error | null = null;
    
    for (let attempt = 0; attempt < (this.config.retries || 1); attempt++) {
      try {
        const controller = new AbortController();
        const timeout = setTimeout(() => controller.abort(), this.config.timeout);
        
        const response = await fetch(new URL(path, this.config.baseUrl), {
          method: 'POST',
          headers: {
            'Content-Type': 'application/json',
            ...this.config.headers,
          },
          body: JSON.stringify(body),
          signal: controller.signal,
        });
        
        clearTimeout(timeout);
        
        if (!response.ok) {
          throw new ServiceError(response.status, await response.text());
        }
        
        const data = await response.json();
        return schema.parse(data);
      } catch (error) {
        lastError = error as Error;
        if (attempt < (this.config.retries || 1) - 1) {
          await sleep(Math.pow(2, attempt) * 1000); // Exponential backoff
        }
      }
    }
    
    throw lastError;
  }
}

// Usage in Next.js API route
// app/api/orders/route.ts
import { z } from 'zod';

const OrderSchema = z.object({
  id: z.string(),
  total: z.number(),
  status: z.enum(['pending', 'paid', 'shipped']),
});

const catalogClient = new ServiceHttpClient({
  baseUrl: process.env.CATALOG_SERVICE_URL!,
  headers: { 'X-API-Key': process.env.CATALOG_API_KEY! },
});

const paymentClient = new ServiceHttpClient({
  baseUrl: process.env.PAYMENT_SERVICE_URL!,
  headers: { 'X-API-Key': process.env.PAYMENT_API_KEY! },
});

export async function POST(request: Request) {
  const body = await request.json();
  
  // Saga pattern: Validate inventory first
  const product = await catalogClient.get(
    `/products/${body.productId}`,
    z.object({ available: z.boolean(), price: z.number() })
  );
  
  if (!product.available) {
    return Response.json({ error: 'Product out of stock' }, { status: 400 });
  }
  
  // Create payment intent
  const payment = await paymentClient.post(
    '/intents',
    { amount: product.price * 100, currency: 'usd' },
    z.object({ clientSecret: z.string(), id: z.string() })
  );
  
  // Create local order record
  const order = await prisma.order.create({
    data: {
      productId: body.productId,
      paymentIntentId: payment.id,
      amount: product.price,
      status: 'pending',
    },
  });
  
  return Response.json(order);
}
```

### gRPC for High-Performance Communication

Binary protocol for internal service mesh:

```typescript
// lib/microservices/grpc/client.ts
import * as grpc from '@grpc/grpc-js';
import * as protoLoader from '@grpc/proto-loader';
import { promisify } from 'util';

// Load protobuf definitions
const packageDefinition = protoLoader.loadSync('./proto/catalog.proto', {
  keepCase: true,
  longs: String,
  enums: String,
  defaults: true,
  oneofs: true,
});

const catalogProto = grpc.loadPackageDefinition(packageDefinition).catalog;

// Type-safe gRPC client wrapper
export class CatalogGrpcClient {
  private client: any;
  
  constructor(address: string) {
    this.client = new (catalogProto as any).CatalogService(
      address,
      grpc.credentials.createInsecure()
    );
  }
  
  async getProduct(sku: string): Promise<{
    sku: string;
    name: string;
    price: number;
    stock: number;
  }> {
    const getProduct = promisify(this.client.getProduct).bind(this.client);
    
    try {
      const response = await getProduct({ sku });
      return {
        sku: response.sku,
        name: response.name,
        price: response.price,
        stock: response.stock_level,
      };
    } catch (error) {
      throw new GrpcError(error.code, error.message);
    }
  }
}

// Usage in API route (requires Node.js runtime)
// app/api/inventory/check/route.ts
export const runtime = 'nodejs'; // gRPC requires Node.js

export async function POST(request: Request) {
  const { sku } = await request.json();
  
  const client = new CatalogGrpcClient(process.env.CATALOG_GRPC_URL!);
  const product = await client.getProduct(sku);
  
  return Response.json({
    available: product.stock > 0,
    stock: product.stock,
  });
}
```

### Asynchronous Messaging

Event-driven communication via message brokers:

```typescript
// lib/microservices/messaging/publisher.ts
import { SNSClient, PublishCommand } from '@aws-sdk/client-sns';
import { EventBridgeClient, PutEventsCommand } from '@aws-sdk/client-eventbridge';

interface DomainEvent {
  eventId: string;
  eventType: string;
  aggregateId: string;
  aggregateType: string;
  timestamp: string;
  payload: unknown;
  metadata: {
    correlationId: string;
    causationId: string;
    userId?: string;
  };
}

export class EventPublisher {
  private sns: SNSClient;
  private eventBridge: EventBridgeClient;
  private topicArn: string;
  
  constructor() {
    this.sns = new SNSClient({ region: process.env.AWS_REGION });
    this.eventBridge = new EventBridgeClient({ region: process.env.AWS_REGION });
    this.topicArn = process.env.EVENTS_TOPIC_ARN!;
  }
  
  async publish(event: Omit<DomainEvent, 'eventId' | 'timestamp'>): Promise<void> {
    const fullEvent: DomainEvent = {
      ...event,
      eventId: crypto.randomUUID(),
      timestamp: new Date().toISOString(),
    };
    
    // Publish to SNS for fan-out
    await this.sns.send(new PublishCommand({
      TopicArn: this.topicArn,
      Message: JSON.stringify(fullEvent),
      MessageAttributes: {
        EventType: {
          DataType: 'String',
          StringValue: event.eventType,
        },
        AggregateType: {
          DataType: 'String',
          StringValue: event.aggregateType,
        },
      },
    }));
    
    // Also publish to EventBridge for routing rules
    await this.eventBridge.send(new PutEventsCommand({
      Entries: [{
        Source: 'order-service',
        DetailType: event.eventType,
        Detail: JSON.stringify(fullEvent),
        EventBusName: 'default',
      }],
    }));
  }
}

// Consumer implementation (Lambda or SQS worker)
// lib/microservices/messaging/consumer.ts
import { SQSHandler } from 'aws-lambda';
import { processOrderCreated } from './handlers/order-created';

export const handler: SQSHandler = async (event) => {
  for (const record of event.Records) {
    const message = JSON.parse(record.body);
    const eventData = JSON.parse(message.Message || record.body);
    
    // Idempotency check
    const processed = await checkIdempotency(eventData.eventId);
    if (processed) {
      console.log(`Event ${eventData.eventId} already processed`);
      continue;
    }
    
    try {
      switch (eventData.eventType) {
        case 'OrderCreated':
          await processOrderCreated(eventData);
          break;
        case 'PaymentProcessed':
          await processPaymentProcessed(eventData);
          break;
        default:
          console.log(`Unhandled event type: ${eventData.eventType}`);
      }
      
      // Mark as processed
      await markAsProcessed(eventData.eventId);
    } catch (error) {
      console.error(`Failed to process event ${eventData.eventId}:`, error);
      // DLQ handling
      throw error;
    }
  }
};
```

## 51.3 API Gateway Integration

Backend-for-Frontend (BFF) pattern with Next.js.

### BFF Architecture

Aggregating microservices for frontend consumption:

```typescript
// app/api/bff/dashboard/route.ts
import { getServerSession } from 'next-auth';
import { authOptions } from '@/app/api/auth/[...nextauth]/route';

// Aggregate data from multiple services for dashboard
export async function GET(request: Request) {
  const session = await getServerSession(authOptions);
  if (!session?.user) {
    return new Response('Unauthorized', { status: 401 });
  }
  
  const userId = session.user.id;
  
  // Parallel fetching from multiple microservices
  const [userProfile, recentOrders, notifications, recommendations] = await Promise.all([
    fetchUserService(userId),      // User service
    fetchOrderService(userId),     // Order service
    fetchNotificationService(userId), // Notification service
    fetchRecommendationService(userId), // ML service
  ]);
  
  // Aggregate and transform for frontend needs
  const dashboardData = {
    user: {
      name: userProfile.name,
      avatar: userProfile.avatar,
      memberSince: userProfile.createdAt,
    },
    stats: {
      totalOrders: recentOrders.total,
      pendingOrders: recentOrders.items.filter(o => o.status === 'pending').length,
      totalSpent: recentOrders.items.reduce((sum, o) => sum + o.total, 0),
    },
    recentActivity: [
      ...recentOrders.items.slice(0, 5).map(order => ({
        type: 'order',
        title: `Order ${order.id}`,
        date: order.createdAt,
        status: order.status,
      })),
      ...notifications.slice(0, 5).map(notif => ({
        type: 'notification',
        title: notif.title,
        date: notif.createdAt,
        read: notif.read,
      })),
    ].sort((a, b) => new Date(b.date).getTime() - new Date(a.date).getTime()),
    recommendations: recommendations.items.map(rec => ({
      id: rec.productId,
      name: rec.name,
      price: rec.price,
      image: rec.imageUrl,
    })),
  };
  
  return Response.json(dashboardData);
}

// GraphQL BFF alternative
// app/api/graphql/route.ts
import { createYoga } from 'graphql-yoga';
import { schema } from '@/lib/graphql/schema';

const yoga = createYoga({
  schema,
  graphqlEndpoint: '/api/graphql',
  context: async ({ request }) => {
    // Auth context
    const token = request.headers.get('authorization')?.split(' ')[1];
    const user = token ? await verifyToken(token) : null;
    
    // Service clients
    return {
      user,
      services: {
        catalog: new CatalogServiceAdapter(),
        orders: new OrderServiceAdapter(),
        payments: new PaymentServiceAdapter(),
      },
    };
  },
});

export { yoga as GET, yoga as POST };
```

### GraphQL Schema Stitching

Federated GraphQL gateway:

```typescript
// lib/graphql/schema.ts
import { makeExecutableSchema } from '@graphql-tools/schema';
import { stitchSchemas } from '@graphql-tools/stitch';
import { buildHTTPExecutor } from '@graphql-tools/executor-http';

// Remote schema executors
const catalogExecutor = buildHTTPExecutor({
  endpoint: `${process.env.CATALOG_SERVICE_URL}/graphql`,
  headers: { 'X-API-Key': process.env.CATALOG_API_KEY! },
});

const orderExecutor = buildHTTPExecutor({
  endpoint: `${process.env.ORDER_SERVICE_URL}/graphql`,
  headers: { 'X-API-Key': process.env.ORDER_API_KEY! },
});

// Stitch schemas from microservices
export const schema = stitchSchemas({
  subschemas: [
    {
      schema: await fetchRemoteSchema(catalogExecutor),
      executor: catalogExecutor,
      merge: {
        Product: {
          fieldName: 'product',
          args: (id) => ({ id }),
          selectionSet: '{ id }',
        },
      },
    },
    {
      schema: await fetchRemoteSchema(orderExecutor),
      executor: orderExecutor,
      merge: {
        Order: {
          fieldName: 'order',
          args: (id) => ({ id }),
        },
      },
    },
  ],
  typeDefs: `
    extend type Product {
      orders: [Order]
    }
    extend type Order {
      product: Product
    }
  `,
  resolvers: {
    Product: {
      orders: {
        resolve(product, _args, context, info) {
          return delegateToSchema({
            schema: orderSchema,
            operation: 'query',
            fieldName: 'ordersByProduct',
            args: { productId: product.id },
            context,
            info,
          });
        },
      },
    },
  },
});
```

## 51.4 Authentication Across Services

Securing microservice communications.

### JWT Token Propagation

Passing authentication context through service calls:

```typescript
// lib/auth/service-auth.ts
import { jwtVerify, SignJWT } from 'jose';

interface ServiceTokenPayload {
  sub: string; // Service identity
  scopes: string[]; // Allowed operations
  iat: number;
  exp: number;
}

// Generate service-to-service token (m2m)
export async function generateServiceToken(
  serviceName: string,
  scopes: string[]
): Promise<string> {
  const secret = new TextEncoder().encode(process.env.SERVICE_JWT_SECRET);
  
  return new SignJWT({
    sub: serviceName,
    scopes,
  })
    .setProtectedHeader({ alg: 'HS256' })
    .setIssuedAt()
    .setExpirationTime('5m') // Short-lived
    .sign(secret);
}

// Verify incoming service token
export async function verifyServiceToken(token: string): Promise<ServiceTokenPayload> {
  const secret = new TextEncoder().encode(process.env.SERVICE_JWT_SECRET);
  
  const { payload } = await jwtVerify(token, secret);
  return payload as ServiceTokenPayload;
}

// HTTP Client with automatic token injection
export class AuthenticatedHttpClient extends ServiceHttpClient {
  private serviceName: string;
  private scopes: string[];
  private cachedToken: string | null = null;
  private tokenExpiry: number = 0;
  
  constructor(config: HttpClientConfig, serviceName: string, scopes: string[]) {
    super(config);
    this.serviceName = serviceName;
    this.scopes = scopes;
  }
  
  private async getValidToken(): Promise<string> {
    const now = Date.now();
    
    // Refresh if expiring in 30 seconds
    if (!this.cachedToken || this.tokenExpiry - now < 30000) {
      this.cachedToken = await generateServiceToken(this.serviceName, this.scopes);
      // Decode to get expiry (simplified)
      this.tokenExpiry = now + 5 * 60 * 1000; // 5 minutes
    }
    
    return this.cachedToken;
  }
  
  async get<T extends z.ZodType>(path: string, schema: T): Promise<z.infer<T>> {
    const token = await this.getValidToken();
    
    return super.get(path, schema, {
      headers: {
        ...this.config.headers,
        'Authorization': `Bearer ${token}`,
      },
    });
  }
}

// Middleware to validate service tokens
// middleware.ts (or API route middleware)
export async function validateServiceAuth(request: Request) {
  const authHeader = request.headers.get('authorization');
  
  if (!authHeader?.startsWith('Bearer ')) {
    return new Response('Missing token', { status: 401 });
  }
  
  const token = authHeader.split(' ')[1];
  
  try {
    const payload = await verifyServiceToken(token);
    
    // Check required scope for this endpoint
    const requiredScope = getRequiredScope(request.url);
    if (!payload.scopes.includes(requiredScope)) {
      return new Response('Insufficient scope', { status: 403 });
    }
    
    // Attach to request for downstream use
    return { service: payload.sub, scopes: payload.scopes };
  } catch (error) {
    return new Response('Invalid token', { status: 401 });
  }
}
```

### API Key Management

Alternative authentication for external clients:

```typescript
// lib/auth/api-keys.ts
import { createHash, randomBytes } from 'crypto';

interface ApiKey {
  id: string;
  keyHash: string;
  name: string;
  scopes: string[];
  rateLimit: number;
  expiresAt?: Date;
  lastUsedAt?: Date;
}

export class ApiKeyManager {
  // Generate new API key (returns key once)
  async generateKey(
    name: string,
    scopes: string[],
    options?: { expiresInDays?: number; rateLimit?: number }
  ): Promise<{ id: string; key: string }> {
    const keyId = randomBytes(16).toString('hex');
    const keySecret = randomBytes(32).toString('base64url');
    const fullKey = `${keyId}.${keySecret}`;
    
    // Store hash only
    const keyHash = createHash('sha256')
      .update(fullKey)
      .digest('hex');
    
    await prisma.apiKey.create({
      data: {
        id: keyId,
        keyHash,
        name,
        scopes,
        rateLimit: options?.rateLimit ?? 1000,
        expiresAt: options?.expiresInDays 
          ? new Date(Date.now() + options.expiresInDays * 86400000)
          : null,
      },
    });
    
    return { id: keyId, key: fullKey };
  }
  
  // Validate API key
  async validateKey(key: string): Promise<ApiKey | null> {
    const [keyId, secret] = key.split('.');
    if (!keyId || !secret) return null;
    
    const keyHash = createHash('sha256')
      .update(key)
      .digest('hex');
    
    const apiKey = await prisma.apiKey.findFirst({
      where: {
        id: keyId,
        keyHash,
        OR: [
          { expiresAt: null },
          { expiresAt: { gt: new Date() } },
        ],
      },
    });
    
    if (apiKey) {
      // Update last used
      await prisma.apiKey.update({
        where: { id: keyId },
        data: { lastUsedAt: new Date() },
      });
    }
    
    return apiKey;
  }
  
  // Revoke key
  async revokeKey(keyId: string): Promise<void> {
    await prisma.apiKey.delete({ where: { id: keyId } });
  }
}
```

## 51.5 Data Synchronization

Managing consistency across distributed services.

### Saga Pattern Implementation

Orchestrating distributed transactions:

```typescript
// lib/sagas/order-saga.ts
type SagaStep = {
  name: string;
  execute: () => Promise<void>;
  compensate: () => Promise<void>;
};

export class OrderSaga {
  private steps: SagaStep[] = [];
  private executedSteps: SagaStep[] = [];
  
  addStep(step: SagaStep) {
    this.steps.push(step);
    return this;
  }
  
  async execute(): Promise<void> {
    for (const step of this.steps) {
      try {
        await step.execute();
        this.executedSteps.push(step);
      } catch (error) {
        // Compensate in reverse order
        for (const executed of this.executedSteps.reverse()) {
          try {
            await executed.compensate();
          } catch (compError) {
            // Log compensation failure - requires manual intervention
            console.error(`Compensation failed for ${executed.name}:`, compError);
          }
        }
        throw error;
      }
    }
  }
}

// Usage in order creation
// app/api/orders/route.ts
export async function POST(request: Request) {
  const body = await request.json();
  
  const saga = new OrderSaga();
  
  let orderId: string;
  let paymentIntentId: string;
  let inventoryReserved = false;
  
  saga
    .addStep({
      name: 'create-order',
      execute: async () => {
        const order = await prisma.order.create({
          data: {
            userId: body.userId,
            status: 'pending',
            items: body.items,
          },
        });
        orderId = order.id;
      },
      compensate: async () => {
        if (orderId) {
          await prisma.order.update({
            where: { id: orderId },
            data: { status: 'cancelled' },
          });
        }
      },
    })
    .addStep({
      name: 'reserve-inventory',
      execute: async () => {
        await fetch(`${process.env.INVENTORY_URL}/reserve`, {
          method: 'POST',
          body: JSON.stringify({ orderId, items: body.items }),
        });
        inventoryReserved = true;
      },
      compensate: async () => {
        if (inventoryReserved) {
          await fetch(`${process.env.INVENTORY_URL}/release`, {
            method: 'POST',
            body: JSON.stringify({ orderId }),
          });
        }
      },
    })
    .addStep({
      name: 'create-payment',
      execute: async () => {
        const payment = await fetch(`${process.env.PAYMENT_URL}/intents`, {
          method: 'POST',
          body: JSON.stringify({
            orderId,
            amount: calculateTotal(body.items),
          }),
        }).then(r => r.json());
        paymentIntentId = payment.id;
      },
      compensate: async () => {
        if (paymentIntentId) {
          await fetch(`${process.env.PAYMENT_URL}/intents/${paymentIntentId}/cancel`, {
            method: 'POST',
          });
        }
      },
    });
  
  try {
    await saga.execute();
    return Response.json({ orderId, status: 'created' });
  } catch (error) {
    return Response.json(
      { error: 'Failed to create order', details: (error as Error).message },
      { status: 500 }
    );
  }
}
```

### Event Sourcing

Audit trail and state reconstruction:

```typescript
// lib/event-sourcing/store.ts
interface Event {
  id: string;
  aggregateId: string;
  aggregateType: string;
  version: number;
  type: string;
  payload: unknown;
  metadata: {
    timestamp: string;
    userId?: string;
    correlationId: string;
  };
}

export class EventStore {
  async append(event: Omit<Event, 'id' | 'version'>): Promise<Event> {
    // Optimistic concurrency control
    const lastEvent = await prisma.event.findFirst({
      where: { aggregateId: event.aggregateId },
      orderBy: { version: 'desc' },
    });
    
    const version = (lastEvent?.version ?? 0) + 1;
    
    return prisma.event.create({
      data: {
        ...event,
        id: crypto.randomUUID(),
        version,
      },
    });
  }
  
  async getEvents(aggregateId: string, fromVersion?: number): Promise<Event[]> {
    return prisma.event.findMany({
      where: {
        aggregateId,
        ...(fromVersion && { version: { gt: fromVersion } }),
      },
      orderBy: { version: 'asc' },
    });
  }
  
  // Project events into current state
  async project<T>(aggregateId: string, reducer: (state: T, event: Event) => T, initialState: T): Promise<T> {
    const events = await this.getEvents(aggregateId);
    return events.reduce(reducer, initialState);
  }
}

// Usage
const eventStore = new EventStore();

// Append event
await eventStore.append({
  aggregateId: orderId,
  aggregateType: 'Order',
  type: 'OrderShipped',
  payload: { trackingNumber: '12345', carrier: 'UPS' },
  metadata: {
    timestamp: new Date().toISOString(),
    userId: currentUser.id,
    correlationId: requestId,
  },
});

// Reconstruct state
const orderState = await eventStore.project(
  orderId,
  (state, event) => {
    switch (event.type) {
      case 'OrderCreated':
        return { ...state, ...event.payload, status: 'created' };
      case 'OrderPaid':
        return { ...state, status: 'paid' };
      case 'OrderShipped':
        return { ...state, status: 'shipped', ...event.payload };
      default:
        return state;
    }
  },
  { status: 'unknown', items: [], total: 0 }
);
```

## 51.6 Error Handling

Resilience patterns for distributed failures.

### Circuit Breaker Pattern

Preventing cascade failures:

```typescript
// lib/resilience/circuit-breaker.ts
type CircuitState = 'CLOSED' | 'OPEN' | 'HALF_OPEN';

interface CircuitBreakerOptions {
  failureThreshold: number;
  resetTimeout: number;
  halfOpenMaxCalls: number;
}

class CircuitBreaker {
  private state: CircuitState = 'CLOSED';
  private failures = 0;
  private lastFailureTime?: number;
  private halfOpenCalls = 0;
  
  constructor(
    private options: CircuitBreakerOptions,
    private fallback?: () => Promise<any>
  ) {}
  
  async execute<T>(fn: () => Promise<T>): Promise<T> {
    if (this.state === 'OPEN') {
      if (Date.now() - (this.lastFailureTime || 0) > this.options.resetTimeout) {
        this.state = 'HALF_OPEN';
        this.halfOpenCalls = 0;
      } else {
        throw new CircuitOpenError('Circuit breaker is OPEN');
      }
    }
    
    if (this.state === 'HALF_OPEN' && this.halfOpenCalls >= this.options.halfOpenMaxCalls) {
      throw new CircuitOpenError('Circuit breaker is HALF_OPEN and max calls reached');
    }
    
    if (this.state === 'HALF_OPEN') {
      this.halfOpenCalls++;
    }
    
    try {
      const result = await fn();
      this.onSuccess();
      return result;
    } catch (error) {
      this.onFailure();
      if (this.fallback) {
        return this.fallback();
      }
      throw error;
    }
  }
  
  private onSuccess() {
    this.failures = 0;
    this.state = 'CLOSED';
  }
  
  private onFailure() {
    this.failures++;
    this.lastFailureTime = Date.now();
    
    if (this.failures >= this.options.failureThreshold) {
      this.state = 'OPEN';
    }
  }
}

// Usage
const catalogBreaker = new CircuitBreaker(
  {
    failureThreshold: 5,
    resetTimeout: 30000,
    halfOpenMaxCalls: 3,
  },
  async () => {
    // Fallback to cache
    return getCachedCatalog();
  }
);

export async function getProduct(sku: string) {
  return catalogBreaker.execute(() => catalogService.getProduct(sku));
}
```

### Retry with Exponential Backoff

Handling transient failures:

```typescript
// lib/resilience/retry.ts
interface RetryOptions {
  maxAttempts: number;
  initialDelay: number;
  maxDelay: number;
  backoffMultiplier: number;
  retryableErrors?: string[];
}

export async function withRetry<T>(
  fn: () => Promise<T>,
  options: RetryOptions
): Promise<T> {
  let attempt = 1;
  let delay = options.initialDelay;
  
  while (true) {
    try {
      return await fn();
    } catch (error: any) {
      if (attempt >= options.maxAttempts) {
        throw error;
      }
      
      // Check if error is retryable
      if (options.retryableErrors) {
        const errorCode = error.code || error.name;
        if (!options.retryableErrors.includes(errorCode)) {
          throw error;
        }
      }
      
      // Wait with jitter
      const jitter = Math.random() * 100;
      await sleep(delay + jitter);
      
      // Exponential backoff
      delay = Math.min(delay * options.backoffMultiplier, options.maxDelay);
      attempt++;
    }
  }
}

// Usage
const result = await withRetry(
  () => paymentService.charge(cardToken, amount),
  {
    maxAttempts: 3,
    initialDelay: 1000,
    maxDelay: 10000,
    backoffMultiplier: 2,
    retryableErrors: ['ECONNRESET', 'ETIMEDOUT', '503'],
  }
);
```

## 51.7 Monitoring Distributed Systems

Observability across service boundaries.

### Distributed Tracing

Correlating requests across services:

```typescript
// lib/observability/tracing.ts
import { trace, context, SpanStatusCode, propagation } from '@opentelemetry/api';

const tracer = trace.getTracer('nextjs-bff');

export async function traceServiceCall<T>(
  serviceName: string,
  operation: string,
  fn: () => Promise<T>,
  parentContext?: any
): Promise<T> {
  // Extract context from incoming request if present
  if (parentContext) {
    context.with(propagation.extract(context.active(), parentContext), async () => {
      // Continue with child span
    });
  }
  
  return tracer.startActiveSpan(`${serviceName}.${operation}`, async (span) => {
    try {
      span.setAttribute('service.name', serviceName);
      span.setAttribute('operation', operation);
      
      const result = await fn();
      
      span.setStatus({ code: SpanStatusCode.OK });
      return result;
    } catch (error: any) {
      span.setStatus({
        code: SpanStatusCode.ERROR,
        message: error.message,
      });
      span.recordException(error);
      throw error;
    } finally {
      span.end();
    }
  });
}

// Propagate context in HTTP headers
export function injectTracingHeaders(headers: Record<string, string> = {}): Record<string, string> {
  propagation.inject(context.active(), headers);
  return headers;
}

// Usage in service client
export async function fetchWithTracing(
  url: string,
  options: RequestInit = {}
): Promise<Response> {
  const headers = injectTracingHeaders(options.headers as Record<string, string>);
  
  return fetch(url, {
    ...options,
    headers: {
      ...headers,
      ...options.headers,
    },
  });
}
```

### Health Checks and Readiness

Service mesh integration:

```typescript
// app/api/health/route.ts
import { prisma } from '@/lib/db';
import { redis } from '@/lib/redis';

export const dynamic = 'force-dynamic';

export async function GET() {
  const checks = await Promise.all([
    checkDatabase(),
    checkRedis(),
    checkExternalServices(),
  ]);
  
  const allHealthy = checks.every(c => c.status === 'healthy');
  
  return Response.json(
    {
      status: allHealthy ? 'healthy' : 'unhealthy',
      timestamp: new Date().toISOString(),
      version: process.env.VERCEL_GIT_COMMIT_SHA || 'unknown',
      checks: Object.fromEntries(checks.map(c => [c.name, c])),
    },
    { status: allHealthy ? 200 : 503 }
  );
}

async function checkDatabase() {
  try {
    await prisma.$queryRaw`SELECT 1`;
    return { name: 'database', status: 'healthy', latency: 0 };
  } catch (error) {
    return { name: 'database', status: 'unhealthy', error: (error as Error).message };
  }
}

async function checkExternalServices() {
  // Check critical dependencies
  const services = [
    { name: 'catalog', url: process.env.CATALOG_HEALTH_URL! },
    { name: 'payment', url: process.env.PAYMENT_HEALTH_URL! },
  ];
  
  return Promise.all(
    services.map(async (svc) => {
      try {
        const start = Date.now();
        const res = await fetch(svc.url, { method: 'HEAD' });
        return {
          name: svc.name,
          status: res.ok ? 'healthy' : 'degraded',
          latency: Date.now() - start,
        };
      } catch (error) {
        return {
          name: svc.name,
          status: 'unhealthy',
          error: (error as Error).message,
        };
      }
    })
  );
}
```

## Key Takeaways from Chapter 51

1. **Domain Decomposition**: Split monoliths using bounded contexts aligned with business capabilities (Catalog, Orders, Payments). Implement Anti-Corruption Layers (ACL) to isolate external service schemas from internal domain models, preventing tight coupling between microservices and Next.js frontend code.

2. **Inter-Service Communication**: Use synchronous HTTP/gRPC for real-time operations requiring immediate consistency (payment authorization), and asynchronous messaging (SNS/SQS/EventBridge) for eventual consistency (inventory updates, notifications). Implement runtime schema validation using Zod to prevent schema drift between services.

3. **Backend-for-Frontend (BFF)**: Aggregate multiple microservice calls in Next.js API routes to reduce frontend chattiness, parallelizing requests with `Promise.all()`. Consider GraphQL federation for complex domain aggregation, stitching schemas from Catalog, Order, and User services into a unified graph.

4. **Service Authentication**: Implement machine-to-machine (M2M) JWT tokens with short expiration (5 minutes) for service-to-service calls, distinct from user session tokens. Use API keys with hashed storage for external client access, validating scopes at the gateway level before forwarding to downstream services.

5. **Distributed Transactions**: Apply Saga pattern for multi-step business processes (Order → Payment → Inventory), implementing compensation logic for each step to maintain eventual consistency during failures. Use event sourcing for audit-critical domains, storing immutable event logs to reconstruct aggregate state at any point in time.

6. **Resilience Patterns**: Deploy circuit breakers to prevent cascade failures when Catalog or Payment services degrade, falling back to cached data or queued processing. Configure exponential backoff with jitter for transient error retries (connection resets, 503 responses), capping maximum delay to prevent thundering herds.

7. **Observability**: Propagate correlation IDs and OpenTelemetry trace context across service boundaries via HTTP headers to trace request paths through distributed systems. Implement comprehensive health checks that verify database connectivity, cache availability, and critical downstream service status for load balancer health determination.

## Coming Up Next

**Chapter 52: Advanced Performance Techniques**

With microservices handling business logic distribution, Chapter 52 focuses on maximizing frontend performance. You'll explore advanced rendering optimization strategies, memory leak prevention in long-running applications, bundle analysis and tree-shaking techniques, network optimization with resource hints, and CPU profiling for JavaScript execution. Learn to leverage the React Compiler for automatic memoization and implement virtual scrolling for massive datasets.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='50. serverless_architecture.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='52. advanced_performance_techniques.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
